<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import pandas as pd 
import plotly.graph_objects as go  
import numpy as np 
from lin_plots import * 


pd.set_option("display.max_rows",
              10000, 
              "display.max_columns", 
              10000)



In [2]:
## Read in the data
offense_stats = pd.read_csv('csv_files/game_predictions_offense.csv') 
defense_stats = pd.read_csv('csv_files/game_predictions_defense.csv') 
offense_defense_stats = pd.read_csv('csv_files/game_predictions_total.csv') 
offense_stats.head()

,Unnamed: 0,team,opponent,year,team_win,unnamed: 0,g,mp,fg,fg%,3p,3p%,ft,ft%,orb,drb,trb,ast,stl,blk,tov,pf,pts,playoffs,unnamed: 0.1,g.1,mp.1,fg.1,fg%.1,3p.1,3p%.1,ft.1,ft%.1,orb.1,drb.1,trb.1,ast.1,stl.1,blk.1,tov.1,pf.1,pts.1,playoffs.1
0,0,atlanta_hawks,boston_celtics,2000,2,24,82.0,241.8,36.6,0.441,3.1,0.317,18.0,0.743,14.0,31.3,45.3,18.9,6.1,5.6,15.4,21.0,94.3,0,8,82.0,240.6,37.2,0.444,5.1,0.331,19.8,0.745,13.5,29.5,43.0,21.2,9.7,3.5,15.4,27.1,99.3,0
1,1,atlanta_hawks,charlotte_hornets,2000,2,24,82.0,241.8,36.6,0.441,3.1,0.317,18.0,0.743,14.0,31.3,45.3,18.9,6.1,5.6,15.4,21.0,94.3,0,13,82.0,241.2,35.8,0.449,4.1,0.339,22.7,0.758,10.8,32.1,42.9,24.7,8.9,5.9,14.7,20.4,98.4,1
2,2,boston_celtics,charlotte_hornets,2000,2,8,82.0,240.6,37.2,0.444,5.1,0.331,19.8,0.745,13.5,29.5,43.0,21.2,9.7,3.5,15.4,27.1,99.3,0,13,82.0,241.2,35.8,0.449,4.1,0.339,22.7,0.758,10.8,32.1,42.9,24.7,8.9,5.9,14.7,20.4,98.4,1
3,3,atlanta_hawks,chicago_bulls,2000,0,24,82.0,241.8,36.6,0.441,3.1,0.317,18.0,0.743,14.0,31.3,45.3,18.9,6.1,5.6,15.4,21.0,94.3,0,28,82.0,241.5,31.3,0.415,4.1,0.329,18.1,0.709,12.6,28.3,40.9,20.1,7.9,4.7,19.0,23.3,84.8,0
4,4,boston_celtics,chicago_bulls,2000,1,8,82.0,240.6,37.2,0.444,5.1,0.331,19.8,0.745,13.5,29.5,43.0,21.2,9.7,3.5,15.4,27.1,99.3,0,28,82.0,241.5,31.3,0.415,4.1,0.329,18.1,0.709,12.6,28.3,40.9,20.1,7.9,4.7,19.0,23.3,84.8,0


In [3]:
def ScoringDiff(df): 
    '''We want to look at the betting lines. 
    Can be determined by looking at the scoring difference''' 

    df['score_diff'] = df['pts'] - df['pts.1']

    return df 
    
offense_stats = ScoringDiff(offense_stats) 
defense_stats = ScoringDiff(defense_stats) 

In [4]:
## Normalize the data- normalize by year, we don't need to normalize playoffs
from sklearn.preprocessing import MinMaxScaler

def Normalize(df): 
    
    ## Normalize By Year and then normalize overall
    
    df_group = df.drop(columns = ['team', 'opponent', 'team_win', 
                                  'playoffs', 'playoffs.1', 'unnamed: 0', 'unnamed: 0.1', 'g', 'g.1', 
                                 'mp', 'mp.1', 'score_diff'])
    df_group = df_group.groupby(['year'])
    df_group = df_group.transform(lambda x: (x - x.mean())/(x.std())) 
    #df_group = df_group.drop(columns = ['level_0', 'index'])
    df_group[df_group.columns] = MinMaxScaler().fit_transform(df_group) 
    
    df = pd.concat(
        [df[['team','opponent', 'year', 'team_win', 'score_diff']], df_group, df[['playoffs', 'playoffs.1']]], axis = 1
    )
    
    return df.loc[:, ~df.columns.str.match('Unnamed') ]


offense_stats_normal = Normalize(offense_stats) 
defense_stats_normal = Normalize(defense_stats) 
offense_stats_normal.head() 

,team,opponent,year,team_win,score_diff,fg,fg%,3p,3p%,ft,ft%,orb,drb,trb,ast,stl,blk,tov,pf,pts,fg.1,fg%.1,3p.1,3p%.1,ft.1,ft%.1,orb.1,drb.1,trb.1,ast.1,stl.1,blk.1,tov.1,pf.1,pts.1,playoffs,playoffs.1
0,atlanta_hawks,boston_celtics,2000,2,-5.0,0.404185,0.352163,0.138065,0.260813,0.375532,0.514640,0.659632,0.532165,0.734343,0.204440,0.160732,0.487211,0.556587,0.258071,0.333888,0.560040,0.384880,0.401717,0.253015,0.534968,0.467704,0.620561,0.369784,0.523058,0.285076,0.600834,0.342313,0.494072,0.868795,0.608335,0,0
1,atlanta_hawks,charlotte_hornets,2000,2,-4.1,0.404185,0.352163,0.138065,0.260813,0.375532,0.514640,0.659632,0.532165,0.734343,0.204440,0.160732,0.487211,0.556587,0.258071,0.333888,0.427149,0.451239,0.263849,0.310239,0.863893,0.540557,0.322730,0.715869,0.513422,0.569782,0.481451,0.666479,0.403221,0.124587,0.570576,0,1
2,boston_celtics,charlotte_hornets,2000,2,0.9,0.449942,0.385560,0.428049,0.346813,0.513892,0.525498,0.601418,0.309040,0.510725,0.369585,0.729092,0.225825,0.556587,0.721450,0.509619,0.427149,0.451239,0.263849,0.310239,0.863893,0.540557,0.322730,0.715869,0.513422,0.569782,0.481451,0.666479,0.403221,0.124587,0.570576,0,1
3,atlanta_hawks,chicago_bulls,2000,0,9.5,0.404185,0.352163,0.138065,0.260813,0.375532,0.514640,0.659632,0.532165,0.734343,0.204440,0.160732,0.487211,0.556587,0.258071,0.333888,0.000000,0.000000,0.263849,0.238708,0.342150,0.265957,0.521284,0.210053,0.320715,0.195597,0.332222,0.504396,0.961303,0.446707,0.000000,0,0
4,boston_celtics,chicago_bulls,2000,1,14.5,0.449942,0.385560,0.428049,0.346813,0.513892,0.525498,0.601418,0.309040,0.510725,0.369585,0.729092,0.225825,0.556587,0.721450,0.509619,0.000000,0.000000,0.263849,0.238708,0.342150,0.265957,0.521284,0.210053,0.320715,0.195597,0.332222,0.504396,0.961303,0.446707,0.000000,0,0


In [5]:
## Perform Principal Component Analysis

from sklearn.decomposition import PCA 

pca = PCA().fit(offense_stats_normal.drop(columns = ['team', 'opponent', 'year', 'team_win', 'score_diff'], axis = 1))  

x = np.array(np.arange(0, pca.explained_variance_ratio_.shape[0], 1)) 
y = np.cumsum(pca.explained_variance_ratio_) 

fig = go.Figure(go.Scatter( x = x, 
                    y = y, 
                    mode = 'lines+markers' ))



max_x = np.array(np.where(y < 0.95)).flatten().max()
max_y = y[max_x]

fig.add_vline(x = max_x, annotation_text = '({}, {})'.format(max_x, round(max_y, 3)))

fig.update_layout(title = 'PCA Number Components vs Explained Variance', 
                    xaxis = dict(title = 'Number of Components'), 
                    yaxis = dict(title = 'Explained Variance'))

fig.show() 

We can decompose this into 18 components, representing 94.5% of the variance 

In [6]:
def PCA_DF(df, dep_cols): 

    ## Perform PCA 

    from sklearn.decomposition import PCA 

    ind_df = df.drop(columns = dep_cols, axis = 1) 

    pca = PCA().fit(ind_df) 

    y = np.cumsum(pca.explained_variance_ratio_) 
    x = np.array(np.arange(0, y.shape[0], 1)) 

    max_x = np.array(np.where(y < 0.95)).flatten().max() 
    max_y = y[max_x]

    fig = go.Figure(go.Scatter(x = x, y = y, mode = 'lines+markers')) 

    fig.add_vline(x = max_x, annotation_text = '({}, {})'.format(max_x, round(max_y, 3))) 

    fig.update_layout(title = 'PCA Number Components vs Explained Variance', 
                    xaxis = dict(title = 'Number of Components'), 
                    yaxis = dict(title = 'Explained Variance'))

    fig.show() 


    pca_df = pd.DataFrame(PCA(n_components = max_x).fit_transform(ind_df))
 

    pca_df[dep_cols] = df[dep_cols] 

    return pca_df, list(range(0, max_x))  

pca_offense, offense_pca_comp_num = PCA_DF(offense_stats_normal, ['team', 'opponent', 'year', 'team_win', 'score_diff'])


In [7]:
pca_offense.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,team,opponent,year,team_win,score_diff
0,-0.094036,0.831844,0.077872,0.043882,-0.324125,-0.333522,0.417964,0.010390,-0.128984,-0.125522,-0.137455,0.205689,-0.043931,0.209384,0.112006,-0.040844,-0.062384,-0.007144,atlanta_hawks,boston_celtics,2000,2,-5.0
1,0.788306,0.160966,0.083387,0.022329,-0.370808,-0.177542,0.202134,0.205230,-0.038421,-0.209061,-0.010847,-0.308777,0.061493,0.014095,-0.182526,0.326530,-0.067879,0.156121,atlanta_hawks,charlotte_hornets,2000,2,-4.1
2,0.749077,0.112763,0.167362,-0.109706,0.100220,-0.057296,-0.275061,-0.240780,0.060151,-0.043043,0.218165,-0.225435,0.199459,0.063779,-0.257597,0.273559,-0.024828,0.168268,boston_celtics,charlotte_hornets,2000,2,0.9
3,-0.329269,1.008618,0.243947,-0.492404,-0.576071,-0.023469,0.082222,0.373172,-0.112869,0.023886,-0.210536,0.101124,0.078388,-0.238484,-0.059448,0.164444,-0.110502,0.267157,atlanta_hawks,chicago_bulls,2000,0,9.5
4,-0.368498,0.960415,0.327922,-0.624440,-0.105043,0.096778,-0.394973,-0.072838,-0.014297,0.189903,0.018476,0.184465,0.216354,-0.188800,-0.134518,0.111473,-0.067450,0.279304,boston_celtics,chicago_bulls,2000,1,14.5


In [8]:
def SplitData(df_pca, deps, ind): 

    from sklearn.model_selection import train_test_split 

    x_train, x_test, y_train, y_test = train_test_split(df_pca[deps],df_pca[ind], test_size = 0.25) 

    return x_train, x_test, y_train, y_test 

train_x, test_x, train_y, test_y = SplitData(pca_offense, offense_pca_comp_num, 'score_diff')

print(train_x.shape, train_y.shape) 

(6416, 18) (6416,)


In [9]:
from sklearn import svm 
from sklearn.tree import DecisionTreeRegressor 
from sklearn.linear_model import Ridge, Lasso

from sklearn.metrics import mean_squared_error

def RidgeReg(train_x, train_y, test_x, test_y, alpha):

    ridge = Ridge(alpha = alpha, normalize = True) 

    ridge.fit(train_x, train_y)  
    y_pred = ridge.predict(test_x)

    score = mean_squared_error(y_pred, test_y)   

    return ridge, y_pred, score 



def LassoReg(train_x, train_y, test_x, test_y, alpha): 


    lasso = Lasso(alpha = alpha, normalize = True) 

    lasso.fit(train_x, train_y) 

    y_pred = lasso.predict(test_x) 

    score = mean_squared_error(y_pred, test_y) 

    return lasso, y_pred, score  



In [29]:
potential_alphas = [1e-15, 1e-10, 1e-8, 1e-5,1e-4, 1e-3,1e-2, 1, 5, 10]

lasso_scores = [] 
ridge_scores = [] 
alphas = [] 

for a in potential_alphas: 

    ridge, ridge_y_pred, ridge_score = RidgeReg(train_x, train_y, test_x, test_y, a) 

    lasso, lasso_y_pred, lasso_score =  LassoReg(train_x, train_y, test_x, test_y, a) 

    lasso_scores.append(lasso_score) 
    ridge_scores.append(ridge_score) 
    alphas.append(str(a)) 

scores = pd.DataFrame({'alphas': alphas, 
                        'lasso_scores': lasso_scores, 
                        'ridge_scores': ridge_scores})
scores.head() 

,alphas,lasso_scores,ridge_scores
0,1e-15,1.421945,1.421945
1,1e-10,1.421945,1.421945
2,1e-08,1.421945,1.421945
3,1e-05,1.421946,1.421945
4,0.0001,1.422996,1.421947


In [39]:
fig = go.Figure() 

fig.add_trace(go.Bar(x = scores.alphas, y = scores.ridge_scores, name = 'Ridge Scores'))
fig.add_trace(go.Bar(x = scores.alphas, y = scores.lasso_scores, name = 'Lasso Scores') ) 

fig.show() 

It seems that across the board the ridge regressions perform better than the lasso regressions. we're going to use a lambda of 0.01. 

In [41]:
## Ridge Regression 
ridge, ridge_y_pred, ridge_score = RidgeReg(train_x, train_y, test_x, test_y, 0.01)

print(ridge_score) 

1.425312730924277


In [43]:
ridge.fit(train_x, train_y) 
mean_squared_error(ridge.predict(train_x), train_y)

1.4265647714449186